## 011 データを読み込む

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
uriage_data = pd.read_csv('./input/100knoks/01-02/uriage.csv')
uriage_data.head()

In [ ]:
uriage_data.describe()

In [ ]:
uriage_data.dtypes

In [ ]:
#kokyaku_data = pd.read_excel('./input/100knoks/01-02/kokyaku_daicho.xlsx')
kokyaku_data = pd.read_csv('./input/100knoks/01-02/kokyaku_daicho.csv')
kokyaku_data.head()

## 012 データの揺れを見る

In [ ]:
uriage_data['purchase_date'].head()

In [ ]:
uriage_data['item_name'].head()

In [ ]:
uriage_data['item_price'].head()

## 013 揺れがあるまま集計

In [ ]:
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime("%Y%m")
uriage_data['purchase_month'].head()

In [ ]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res

In [ ]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='sum', fill_value=0)
res

## 014 商品名の揺れを補正

無駄なスペースを削除して、英数字を半角全角、大文字小文字を統一する

In [ ]:
print(len(pd.unique(uriage_data['item_name'])))

In [ ]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
print(len(pd.unique(uriage_data['item_name'])))

In [ ]:
uriage_data['item_name'] = uriage_data['item_name'].str.replace('　', '')
print(len(pd.unique(uriage_data['item_name'])))

In [ ]:
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '')
print(len(pd.unique(uriage_data['item_name'])))

In [ ]:
uriage_data.sort_values(by='item_name', ascending=True)

In [ ]:
print(uriage_data['item_name'].unique())
print(len(uriage_data['item_name'].unique()))

## 015 金額の欠損を補完する

In [ ]:
uriage_data.isnull().sum()

In [ ]:
uriage_data.isnull().any(axis=0)

In [ ]:
flg_is_null = uriage_data['item_price'].isnull()
flg_is_null

In [ ]:
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()
    uriage_data.loc[(flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'] = price

uriage_data.head()

### locメソッドの使い方
locメソッドは、条件を指定してその条件に合致するデータフレームを抽出することが可能。
１つ目に条件を渡す。今回の場合は金額がNaNのデータなので、flag_is_nullを渡す。2番目に取り出すデータのカラム名を渡すので
今回は'item_name'を渡している。

In [ ]:
uriage_data.isnull().any(axis=0)

In [ ]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg, '最大値', uriage_data.loc[uriage_data['item_name'] == trg, 'item_price'].max(skipna=False))
    print(trg, '最小値', uriage_data.loc[uriage_data['item_name'] == trg, 'item_price'].min(skipna=False))
    print('-'*80)

## 016 顧客名の揺れを補正する

In [ ]:
kokyaku_data['顧客名'].head()

In [ ]:
uriage_data['customer_name'].head()

In [ ]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace('　', '')
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ', '')
kokyaku_data.head()


## 017 日付の揺れを補正する

In [ ]:
kokyaku_data['登録日'].astype('str').str.isdigit().sum()

In [ ]:
flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial

In [ ]:
flg_is_jstring = kokyaku_data['登録日'].astype('str').str.find('年') > 0
flg_is_jstring

In [ ]:
flg_is_string = kokyaku_data['登録日'].astype('str').str.find('/') > 0
flg_is_string

In [ ]:
from_serial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float'), unit='D') + pd.to_datetime('1900/01/01')
from_serial

In [ ]:
from_jstring = pd.to_datetime(kokyaku_data.loc[flg_is_jstring, '登録日'], format="%Y年%m月%d日")

In [ ]:
from_string = pd.to_datetime(kokyaku_data.loc[flg_is_string, '登録日'], format="%Y/%m/%d")
from_string

In [ ]:
kokyaku_data['登録日'] = pd.concat([from_jstring, from_string])
kokyaku_data

In [ ]:
kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime("%Y%m")
res = kokyaku_data.groupby('登録年月').count()['顧客名']
print(res)
print(res.sum())

## 018 顧客名をキーにデータを結合 Join

In [ ]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='left')
join_data = join_data.drop('customer_name', axis=1)
join_data

## 019 クレンジングしたデータをダンプする

In [ ]:
dump_data = join_data[join_data.columns.drop('登録年月')]
dump_data

In [ ]:
dump_data.to_csv('./input/100knoks/01-02/dump_data.csv', index=False)

## 020 データを集計する

In [ ]:
import_data = pd.read_csv('./input/100knoks/01-02/dump_data.csv')
import_data

In [ ]:
import_data.describe()

In [ ]:
import_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)

In [ ]:
import_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)

In [ ]:
import_data.pivot_table(index='purchase_month', columns='顧客名', aggfunc='size', fill_value=0)

In [ ]:
import_data.pivot_table(index='purchase_month', columns='地域', aggfunc='size', fill_value=0)

In [ ]:
away_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='right')
away_data

In [ ]:
away_data[away_data['purchase_date'].isnull()]

In [ ]:
away_data[away_data['purchase_date'].isnull()][['顧客名', 'メールアドレス', '登録日']]